# New Crawling Function
---

## 크롤링 방안

| 구분 | 장점 | 단점 |
| :-: | :-: | :-: |
| naver 뉴스 API 활용 | 쉬운 사용 방법, 낮은 구현 난이도 | 수집 뉴스 수량 제한 (1일 최대 25000회 / 1회 최대 100개) |
| 각 언론사 별 뉴스 홈페이지 직접 크롤링 | 뉴스 크롤링 직접 구현 필요 | 수집량 제한 없음 (과도한 요청 지양 필요) |

## 언론사 리스트

- 국민일보
- The JoongAng
- 뉴데일리
- 데일리안
- 한국경제TV
- KBS NEWS
- 아이뉴스24
- 문화일보
- Jtbc
- 시사IN
- ChannelA
- 서울신문
- YTN
- 한겨레
- MBN NEWS
- SBS NEWS
- 해럴드경제
- 매일경제
- 경향신문
- ChosunBiz
- 조선일보

## Import

In [19]:
# selenium framework
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Set Chrome Driver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager # chrome web driver manager

# parse html
from bs4 import BeautifulSoup

# utils
import time
import pandas as pd

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # get web driver
driver.get("https://www.kmib.co.kr/") # get into website url

# 검색 버튼 클릭
search_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div/header/div[1]/nav/div[3]/ul/li[1]/button'))
)
search_btn.click()

# 키워드 검색
search_input_element = driver.find_element(By.XPATH, '/html/body/div/header/div[2]/div/div/div/input')
search_btn = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '/html/body/div/header/div[2]/div/div/div/button'))
)
search_input_element.send_keys("삼성전자")
search_btn.click()
time.sleep(3)

articles = []

# 뉴스 검색 결과 순환 (페이지 내 순환 -> 페이지 순환)
for _ in range(3):
    for page_idx in range(3,7):
        news_entities = driver.find_elements(By.CLASS_NAME, "card_body") # 페이지 내 뉴스 수 파악
        for idx in range(1, len(news_entities)+1):
            # news 접근
            news_entity = driver.find_element(By.XPATH, f"/html/body/div[1]/div/div/section/div[2]/div[{idx}]/div/div[1]/a")
            news_entity.click()
            time.sleep(2)
            
            # 제목 수집
            header = driver.find_element(By.XPATH, "/html/body/div[1]/div/section/article/header/h1").text
            # 게시/수정일 수집
            post_date = driver.find_element(By.XPATH, "/html/body/div[1]/div/section/article/header/div[2]/div[1]/div[1]/span").text
            try: modified_date = driver.find_element(By.XPATH, "/html/body/div[1]/div/section/article/header/div[2]/div[1]/div[2]/span").text
            except: modified_date = None
            # 기사 본문 수집
            article_body = driver.find_element(By.XPATH, "/html/body/div[1]/div/section/article/div[1]/div[1]").text

            # 수집 결과 정리
            articles.append(
                {
                    "header" : header,
                    "meta_data" : {
                        "post_date" : post_date,
                        "modified_date" : modified_date
                    },
                    "content" : article_body
                }
            )

            # 목록으로 복귀
            driver.back()
            time.sleep(1)
        
        # 다음 페이지 이동
        next_page_btn = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"/html/body/div[1]/div/div/section/div[3]/ul/li[{page_idx}]/a"))
        )
        next_page_btn.click()
        time.sleep(1.5)
    
    # 다음 페이지 목록으로 이동
    next_page_nums_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[1]/div/div/section/div[3]/ul/li[7]/a"))
    )
    next_page_nums_btn.click()
    time.sleep(1.5)

In [16]:
len(articles)

120

In [22]:
pd.DataFrame(articles).to_csv("./data/news_crawling_results.csv", index=False)